The Stigler diet is an optimization problem named for George Stigler, a 1982 Nobel laureate in economics, who posed the following problem:

For a moderately active person weighing 154 pounds, how much of each of 77 foods should be eaten on a daily basis  
so that the man’s intake of nine nutrients will be at least equal to the recommended dietary allowances (RDAs)  
suggested by the National Research Council in 1943, with the cost of the diet being minimal?  

source:https://en.wikipedia.org/wiki/Stigler_diet

Load data:

In [2]:
import pandas as pd

commodities_list = pd.read_csv("commodities_list.csv")
nutrients_list = pd.read_csv("nutrients_list.csv")

In [3]:
commodities_list.head()

,Commodity,Unit,1939 price (cents),Calories (kcal),Protein (g),Calcium (g),Iron (mg),Vitamin A (KIU),Thiamine (mg),Riboflavin (mg),Niacin (mg),Ascorbic Acid (mg)
0,Wheat Flour,10 lb.,36.0,44.7,1411,2.0,365,0.0,55.4,33.3,441,0
1,Macaroni,1 lb.,14.1,11.6,418,0.7,54,0.0,3.2,1.9,68,0
2,Wheat Cereal,28 oz.,24.2,11.8,377,14.4,175,0.0,14.4,8.8,114,0
3,Corn Flakes,8 oz.,7.1,11.4,252,0.1,56,0.0,13.5,2.3,68,0
4,Corn Meal,1 lb.,4.6,36.0,897,1.7,99,30.9,17.4,7.9,106,0


In [4]:
nutrients_list.head()

,Nutrient,Daily Recommended Intake
0,Calories,"3,000 calories"
1,Protein,70 grams
2,Calcium,.8 grams
3,Iron,12 milligrams
4,Vitamin A,"5,000 IU"


In [1]:
import xpress as xp

In [5]:
model = xp.problem(name="stigler-diet")

Using the Community license in this session. If you have a full Xpress license, pass the full path to your license file to xpress.init(). If you want to use the FICO Community license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('c:/Users/szymczak.a.4/AppData/Local/Programs/Python/Python311/Lib/site-packages/xpress/license/community-xpauth.xpr')


Create variables

In [9]:
import re
x=[]

for index, row in commodities_list["Commodity"].items():
    name = row.replace(" ", "_")
    temp = model.addVariable(vartype=xp.integer, name=name)
    x.append(temp)


TypeError: problem.addVariable() takes no keyword arguments

In [ ]:
calories_constraint = model.addConstr((gb.quicksum(commodities_list["Calories (kcal)"][idx]*food for idx, food in enumerate(x))) == 3000 , name="meet_calories_restrictions")

In [ ]:
protein_constraint = model.addConstr((gb.quicksum(commodities_list["Protein (g)"][idx]*food for idx, food in enumerate(x))) == 70 , name="meet_protein_restrictions")

In [ ]:
calcium_constraint = model.addConstr((gb.quicksum(commodities_list["Calcium (g)"][idx]*food for idx, food in enumerate(x))) == 0.8 , name="meet_calcium_restrictions")

In [ ]:
iron_constraint = model.addConstr((gb.quicksum(commodities_list["Iron (mg)"][idx]*food for idx, food in enumerate(x))) == 0.8 , name="meet_iron_restrictions")

In [ ]:
vitaminA_constraint = model.addConstr((gb.quicksum(commodities_list["Vitamin A (KIU)"][idx]*food for idx, food in enumerate(x))) == 5000 , name="meet_vitaminA_restrictions")

In [ ]:
vitaminB1_constraint = model.addConstr((gb.quicksum(commodities_list["Thiamine (mg)"][idx]*food for idx, food in enumerate(x))) == 1.8 , name="meet_vitaminB1_restrictions")

In [ ]:
vitaminB2_constraint = model.addConstr((gb.quicksum(commodities_list["Riboflavin (mg)"][idx]*food for idx, food in enumerate(x))) == 2.7 , name="meet_vitaminB2_restrictions")

In [ ]:
niacyn_constraint = model.addConstr((gb.quicksum(commodities_list["Niacin (mg)"][idx]*food for idx, food in enumerate(x))) == 18 , name="meet_niacyn_restrictions")

In [ ]:
model.remove(calories_constraint)

In [ ]:
model.update()

In [ ]:
model.setObjective(gb.quicksum(food*commodities_list["1939 price (cents)"][idx] for idx, food in enumerate(x)), GRB.MINIMIZE)

In [ ]:
model.write('diet_problem.lp')

In [ ]:
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 77 columns and 540 nonzeros
Model fingerprint: 0xf9cec2c9
Coefficient statistics:
  Matrix range     [1e-01, 2e+03]
  Objective range  [4e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-01, 5e+03]
Presolve time: 0.02s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Infeasible model


In [ ]:
x_values = pd.Series(model.getAttr("Wheat_Flour", x), name = "shipment")

AttributeError: 'gurobipy.Model' object has no attribute 'Wheat_Flour'

In [ ]:
x[0]

<gurobi.Var Wheat_Flour>

Create decision variables  
 ~the values that we need to decide on

In [ ]:
decision_var = {(i,j,k): xp.var(name=f"x_{i}_{j}_{k}") for i in range(0,9) for j in range(0,9) for k in range(0,9)}

In [ ]:
problem = xp.problem(gb.quicksum(food*commodities_list["1939 price (cents)"][idx] for idx, food in enumerate(x)), GRB.MINIMIZE)

NameError: name 'xp' is not defined